## **CYK Algorithm - Parsing**

## Import Libraries

In [ ]:
!pip install nltk

In [ ]:
!pip install treelib

  Created wheel for treelib: filename=treelib-1.6.1-cp36-none-any.whl size=18369 sha256=eeba6dbda6585acb0b402c83fa1105cff753b6267c9ae102c841b351b1f93e5d
  Stored in directory: /root/.cache/pip/wheels/68/1d/92/c50ec52951ccebafb40f3b8f0beb28fbaf745431c14a17c497
Successfully built treelib


In [ ]:
import nltk
import json
from nltk import PCFG,Production,nonterminals, Nonterminal
from treelib import Node, Tree
from collections import defaultdict

# 1.Context Free Grammar to Chomsky Normal Form Convertion


In [ ]:
# Global dictionary used for storing the rules.
from treelib import Node, Tree
RULE_DICT = {}


def read_grammar(grammar_file):
    """
    Reads in the given grammar file and splits it into separate lists for each rule.
    :param grammar_file: the grammar file to read in.
    :return: the list of rules.
    """
    with open(grammar_file) as cfg:
        lines = cfg.readlines()
    return [x.replace("->", "").split() for x in lines]


def add_rule(rule):
    """
    Adds a rule to the dictionary of lists of rules.
    :param rule: the rule to add to the dict.
    """
    global RULE_DICT
    
    if rule[0] not in RULE_DICT:
        RULE_DICT[rule[0]] = []
    RULE_DICT[rule[0]].append(rule[1:])


def convert_grammar(grammar):
    """
        Converts a context-free grammar in the form of

        S -> NP VP
        NP -> Det ADV N

        and so on into a chomsky normal form of that grammar. After the conversion rules have either
        exactly one terminal symbol or exactly two non terminal symbols on its right hand side.

        Therefore some new non terminal symbols might be created. These non terminal symbols are
        named like the symbol they replaced with an appended index.
    :param grammar: the CFG.
    :return: the given grammar converted into CNF.
    """

    # Remove all the productions of the type A -> X B C or A -> B a.
    global RULE_DICT
    unit_productions, result = [], []
    res_append = result.append
    index = 0

    for rule in grammar:
        new_rules = []
        if len(rule) == 2 and rule[1][0] != "'":
            # Rule is in form A -> X, so back it up for later and continue with the next rule.
            unit_productions.append(rule)
            add_rule(rule)
            continue
        elif len(rule) > 2:
            # Rule is in form A -> X B C [...] or A -> X a.
            terminals = [(item, i) for i, item in enumerate(rule) if item[0] == "'"]
            if terminals:
                for item in terminals:
                    # Create a new non terminal symbol and replace the terminal symbol with it.
                    # The non terminal symbol derives the replaced terminal symbol.
                    rule[item[1]] = f"{rule[0]}{str(index)}"
                    new_rules += [f"{rule[0]}{str(index)}", item[0]]
                index += 1
            while len(rule) > 3:
                new_rules.append([f"{rule[0]}{str(index)}", rule[1], rule[2]])
                rule = [rule[0]] + [f"{rule[0]}{str(index)}"] + rule[3:]
                index += 1
        # Adds the modified or unmodified (in case of A -> x i.e.) rules.
        add_rule(rule)
        res_append(rule)
        if new_rules:
            result.extend(new_rules)
    # Handle the unit productions (A -> X)
    while unit_productions:
        rule = unit_productions.pop()
        if rule[1] in RULE_DICT:
            for item in RULE_DICT[rule[1]]:
                new_rule = [rule[0]] + item
                if len(new_rule) > 2 or new_rule[1][0] == "'":
                    res_append(new_rule)
                else:
                    unit_productions.append(new_rule)
                add_rule(new_rule)
    return result

with open('grammar.txt') as f:
        grammar = [line.rstrip('\n') for line in f]
grammar = [x.replace("-> ","").split() for x in grammar]

#Rules of the grammar
rules = convert_grammar(grammar)
print(rules)


[['S', 'NP', 'VP'], ['S', 'S0', 'VP'], ['S0', 'NP', 'NP'], ['NP', 'DT', 'NN'], ['NP', 'NP', 'PP'], ['NP', 'NP1', 'NN'], ['NP1', 'DT', 'NN'], ['VP', 'V', 'NP'], ['VP', 'VP2', 'PP'], ['VP2', 'V', 'NP'], ['PP', 'IN', 'NP'], ['DT', "'my'"], ['DT', "'a'"], ['DT', "'the'"], ['DT', "'an'"], ['DT', "'this'"], ['DT', "'that'"], ['NN', "'I'"], ['NN', "'elephant'"], ['NN', "'pajamas'"], ['NN', "'dog'"], ['NN', "'man'"], ['NN', "'burglar'"], ['NN', "'apartment'"], ['NN', "'Boeing'"], ['NN', "'Seattle'"], ['NN', "'IBM'"], ['NN', "'book'"], ['NN', "'orange'"], ['NN', "'he'"], ['NN', "'street'"], ['NN', "'car'"], ['NN', "'park'"], ['NN', "'Smith'"], ['NN', "'Gates'"], ['NN', "'telephone'"], ['NN', "'house'"], ['NN', "'building'"], ['NN', "'box'"], ['NN', "'mechanic'"], ['NN', "'pigeon'"], ['IN', "'in'"], ['IN', "'down'"], ['IN', "'of'"], ['IN', "'out'"], ['IN', "'up'"], ['IN', "'beside'"], ['IN', "'above'"], ['IN', "'as'"], ['IN', "'under'"], ['IN', "'on'"], ['IN', "'with'"], ['V', "'is'"], ['V', "'s

# 2. CYK Recognition

In [ ]:
def recognise_CYK(w,list1):
    # Function to perform the CYK Algorithm 
    def cykParse(w,R): 
    	n = len(w) 
    	
    	# Initialize the table 
    	T = [[set([]) for j in range(n)] for i in range(n)] 
    
    	# Filling in the table 
    	for j in range(0, n): 
    
    		# Iterate over the rules 
            for lhs, rule in R.items(): 
                for rhs in rule: 
                      
                    # If a terminal is found 
                    if len(rhs) == 1 and rhs[0] == w[j]: 
                        T[j][j].add(lhs) 
      
            for i in range(j, -1, -1):    
                   
                # Iterate over the range i to j + 1    
                for k in range(i, j + 1):      
      
                    # Iterate over the rules 
                    for lhs, rule in R.items(): 
                        for rhs in rule: 
                              
                            # If a terminal is found 
                            if len(rhs) == 2 and rhs[0] in T[i][k] and (k < n-1) and rhs[1] in T[k + 1][j]: 
                                T[i][j].add(lhs)
    
    	# If word can be formed by rules 
    	# of given grammar 
    	if len(T[0][n-1]) != 0: 
    		 return "True" 
    	return "False"
    	
    
    R = {}
        
    for i in list1:
        index = 0
        for j in i:
            if index == 0:
                index = j
                if j not in R.keys():
                    R[j]=[]
            else:
                R[index].append([i[x] for x in range(1,len(i))])
                break
    
    # Function Call 
    check = cykParse(w,R) 
    return check




In [ ]:
with open('grammar.txt') as f:
        grammar = [line.rstrip('\n') for line in f]
grammar = [x.replace("-> ","").split() for x in grammar]

#Rules of the grammar
rules = convert_grammar(grammar)
rules = [[x.replace("'","") for x in l] for l in rules]
sentence = "a burglar robbed the apartment"
result = recognise_CYK(sentence.split(),rules)
print(sentence,"--",result)

sentence = "a man is sleeping"
result = recognise_CYK(sentence.split(),rules)
print(sentence,"--",result)

a burglar robbed the apartment -- True
a man is sleeping -- False


# 3. CYK Parsing

In [ ]:
import os.path
import argparse



class Node_:
    """
    Used for storing information about a non-terminal symbol. A node can have a maximum of two
    children because of the CNF of the grammar.
    It is possible though that there are multiple parses of a sentence. In this case information
    about an alternative child is stored in self.child1 or self.child2 (the parser will decide
    where according to the ambiguous rule).
    Either child1 is a terminal symbol passed as string, or both children are Nodes.
    """

    def __init__(self, symbol, child1, child2=None):
        self.symbol = symbol
        self.child1 = child1
        self.child2 = child2

    def __repr__(self):
        """
        :return: the string representation of a Node_ object.
        """
        return self.symbol


class Parser:
    """
    A CYK parser which is able to parse any grammar in CNF. The grammar can be read from a file or
    passed as a string. It either returns a string representation of the parse tree(s) or prints it
    to standard out.
    """

    def __init__(self, grammar, sentence):
        """
        Creates a new parser object which will read in the grammar and transform it into CNF and
        then parse the given sentence with that grammar.
        :param grammar: the file path to the grammar/the string repr. of the grammar to read in
        :param sentence: the file path to the sentence/the string repr. of the sentence to read in
        """
        self.parse_table = None
        self.prods = {}
        self.grammar = None
        if os.path.isfile(grammar):
            self.grammar_from_file(grammar)
        else:
            self.grammar_from_string(grammar)
        self.__call__(sentence)

    def __call__(self, sentence, parse=False):
        """
        Parse the given sentence (string or file) with the earlier given grammar.
        :param sentence: the sentence to parse with self.grammar
        """
        if os.path.isfile(sentence):
            with open(sentence) as inp:
                self.input = inp.readline().split()
                if parse:
                    self.parse()
        else:
            self.input = sentence.split()

    def grammar_from_file(self, grammar):
        """
        Reads in a CFG from a given file, converts it to CNF and stores it in self.grammar.
        :param grammar: the file in which the grammar is stored.
        """
        self.grammar = convert_grammar(read_grammar(grammar))

    def grammar_from_string(self, grammar):
        """
        Reads in a CFG from a string, converts it to CNF and stores it in self.grammar.
        :param grammar: the CFG in string representation.
        :return:
        """
        self.grammar = convert_grammar([x.replace("->", "").split() for x in grammar.split("\n")])

    def parse(self):
        """
        Does the actual parsing according to the CYK algorithm. The parse table is stored in
        self.parse_table.
        """
        length = len(self.input)
        # self.parse_table[y][x] is the list of nodes in the x+1 cell of y+1 row in the table.
        # That cell covers the word below it and y more words after.
        self.parse_table = [[[] for x in range(length - y)] for y in range(length)]

        for i, word in enumerate(self.input):
            # Find out which non terminals can generate the terminals in the input string
            # and put them into the parse table. One terminal could be generated by multiple
            # non terminals, therefore the parse table will contain a list of non terminals.
            for rule in self.grammar:
                if f"'{word}'" == rule[1]:
                    self.parse_table[0][i].append(Node_(rule[0], word))
        for words_to_consider in range(2, length + 1):
            for starting_cell in range(0, length - words_to_consider + 1):
                for left_size in range(1, words_to_consider):
                    right_size = words_to_consider - left_size

                    left_cell = self.parse_table[left_size - 1][starting_cell]
                    right_cell = self.parse_table[right_size - 1][starting_cell + left_size]

                    for rule in self.grammar:
                        left_nodes = [n for n in left_cell if n.symbol == rule[1]]
                        if left_nodes:
                            right_nodes = [n for n in right_cell if n.symbol == rule[2]]
                            self.parse_table[words_to_consider - 1][starting_cell].extend(
                                [Node_(rule[0], left, right) for left in left_nodes for right in right_nodes]
                            )

    def print_tree(self, output=True):
        """
        Print the parse tree starting with the start symbol. Alternatively it returns the string
        representation of the tree(s) instead of printing it.
        """
        start_symbol = self.grammar[0][0]
        final_nodes = [n for n in self.parse_table[-1][0] if n.symbol == start_symbol]
        if final_nodes:
            #print(final_nodes)
            trees = [generate_tree(node) for node in final_nodes]
            print(trees[0])
            root, *tail = trees[0]
            tree = Tree()

            node = Node(root)
            tree.add_node(node)

            q = [[node, *tail]]
            while q:
              parent, *children = q.pop()
              for child in children:
                  if isinstance(child, list):
                      head, *tail = child
                      node = tree.create_node(head, parent=parent)
                      q.append([node, *tail])
                  else:
                      tree.create_node(child, parent=parent)

            tree.show()

            #if output:
             #   for tree in trees:
              #      print(tree)
            #else:
             #   return trees
        
def generate_tree(node):
    """
    Generates the string representation of the parse tree.
    :param node: the root node.
    :return: the parse tree in string form.
    """
    if node.child2 is None:
        return [node.symbol ,node.child1]
    return [node.symbol,generate_tree(node.child1),generate_tree(node.child2)]


if __name__ == '__main__':
    
    grammarP = 'grammar.txt'
    with open('grammar.txt') as f:
        grammar = [line.rstrip('\n') for line in f]
    grammar = [x.replace("-> ","").split() for x in grammar]

    #Rules of the grammar
    rules = convert_grammar(grammar)
    rules = [[x.replace("'","") for x in l] for l in rules]
    sentence = "a burglar robbed the apartment"
    result = recognise_CYK(sentence.split(),rules)
    print(sentence,"--",result)

    
    if(result=="True"):
      sentence = 'sentence.txt'
      print("The given sentence is contained in the language produced by the given grammar!")
      print("\nPossible parse(s):")
      CYK = Parser(grammarP, sentence)
      CYK.parse()
      CYK.print_tree()
    else:
      print("The given sentence is not contained in the language produced by the given grammar!")


a burglar robbed the apartment -- True
The given sentence is contained in the language produced by the given grammar!

Possible parse(s):
['S', ['NP', ['DT', 'a'], ['NN', 'burglar']], ['VP', ['V', 'robbed'], ['NP', ['DT', 'the'], ['NN', 'apartment']]]]
S
├── NP
│   ├── DT
│   │   └── a
│   └── NN
│       └── burglar
└── VP
    ├── NP
    │   ├── DT
    │   │   └── the
    │   └── NN
    │       └── apartment
    └── V
        └── robbed



# 4. Probabilistic CYK

In [ ]:
pcfg = PCFG.fromstring("""S -> NP VP [1.0]
                          NP -> DT NN [0.5]
                          NP -> NP PP [0.25]
                          NP -> 'John' [0.1]
                          NP -> 'I' [0.15]
                          PP -> P NP [1.0]
                          VP -> V [0.4]
                          VP -> Vt NP [0.4]
                          VP -> VP PP [0.2]
                          V -> 'sleeps' [1.0]
                          Vt -> 'saw' [0.7]
                          Vt -> 'ate' [0.3]
                          P -> 'with' [0.7]
                          P -> 'under' [0.3]
                          DT -> 'the' [0.7]
                          DT -> 'a' [0.3]
                          NN -> 'man' [0.7]
                          NN -> 'dog' [0.2]
                          NN -> 'telescope' [0.1]
                        """)

In [ ]:
print(pcfg)

Grammar with 19 productions (start state = S)
    S -> NP VP [1.0]
    NP -> DT NN [0.5]
    NP -> NP PP [0.25]
    NP -> 'John' [0.1]
    NP -> 'I' [0.15]
    PP -> P NP [1.0]
    VP -> V [0.4]
    VP -> Vt NP [0.4]
    VP -> VP PP [0.2]
    V -> 'sleeps' [1.0]
    Vt -> 'saw' [0.7]
    Vt -> 'ate' [0.3]
    P -> 'with' [0.7]
    P -> 'under' [0.3]
    DT -> 'the' [0.7]
    DT -> 'a' [0.3]
    NN -> 'man' [0.7]
    NN -> 'dog' [0.2]
    NN -> 'telescope' [0.1]


In [ ]:
class PCFGParser():
    
    
    def __init__(self,grammar):
        
        #Initialize Nonterminals and the Unary and Binary rules
        
        self.unary_rules = []
        self.binary_rules =[]
        self.N =['S','NP','PP','VP','V','Vt','P','DT','NN']  
        self.rules =[]

        
        for rule in grammar.productions():
            
            self.rules.append(rule)
            
            if len(rule)==1:
                self.unary_rules.append(rule)
            elif len(rule)==2:
                self.binary_rules.append(rule)
    
    def q(self, X , Y, Z):
        
        #Returns probabilities for Binary rules
        
        
        if(Y==Z):
            for rule in self.rules:
                if len(rule)==1:
                    if str(rule.rhs()[0]) in N:
                        return rule.prob()
        
        
        for rule in self.binary_rules:
            if str(rule.lhs())==X and str(rule.rhs()[0])==Y and str(rule.rhs()[1])==Z:
            
                return rule.prob()
            
        return 0

    def q_unary(self, X, W):
        
        #Returns probabilities for Unary rules
        
        for rule in self.unary_rules:
            if str(rule.lhs())==X and rule.rhs()[0]==W:
                return rule.prob()
        return 0
    
    
    def parse(self, sentence):
        
        #Calls the CYK Algorithm and stores the parse tree in a JSON Format
        
        sentence  = sentence.strip()
        parse_output = json.dumps(self.CKY(sentence.split(' ')))
        #print (json.dumps(self.CKY(sentence.split(' '))))
        parse_tree = json.loads(parse_output)
        print(parse_tree)
        
        root, *tail = parse_tree
        tree = Tree()
        node = Node(root)
        tree.add_node(node)

        q = [[node, *tail]]
        while q:
            parent, *children = q.pop()
            for child in children:
                if isinstance(child, list):
                    head, *tail = child
                    node = tree.create_node(head, parent=parent)
                    q.append([node, *tail])
                else:
                    tree.create_node(child, parent=parent)

        tree.show()
        
        
                
    def CKY(self, x):
        
        #Returns Tree for a grammar in Chomsky-Normal Form 
        
        n = len(x) # length of sentence x
        pi = defaultdict(float) # DP table pi
        bp = {} # back pointers

        # Base case
        for i in range(n):
            w = x[i] 
            for X in self.N:
                pi[i, i, X] = self.q_unary(X, w) 
                #Handle Unary rules ending with Non-terminals
                if n <=3:
                    if pi[i,i,X]:
                        for rule in self.rules:
                            if len(rule)==1:
                                if str(rule.rhs()[0])==X:
                                    pi[i,i,str(rule.lhs())] = rule.prob()
        
        # Recursive case
        for l in range(1, n): 
            for i in range(n-l):
                j = i + l
                for X in self.N:
                    max_score = 0
                    args = None
                    for R in self.binary_rules: # search only within the rules with non-zero probability
                        
                        
                        if str(R.lhs()) == X: # consider rules which start from X
                            Y,Z= R.rhs()
                            Y = str(Y)
                            Z = str(Z)
                            for s in range(i, j):
                                        
                                if pi[i, s, Y] and pi[s + 1, j, Z]: # calculate score if both pi entries have non-zero score
                                    score = self.q(X, Y, Z) * pi[i, s, Y] * pi[s + 1, j, Z]
                                    if max_score < score:
                                        max_score = score
                                        args = Y, Z, s
                                            
                                            
                    if max_score: # update table and back pointers
                        pi[i, j, X] = max_score
                        bp[i, j, X] = args

        # Backtrack to retrieve the tree
        if pi[0, n-1, 'S']:
            return self.backtrack_tree(x, bp, 0, n-1, 'S')
        else: # if start symbol is not 'S'
            max_score = 0
            args = None
            for X in self.N:
                print(pi[0, n-1, X])
                if max_score < pi[0, n-1, X]:
                    max_score = pi[0, n-1, X]
                    args = 0, n-1, X
            return self.backtrack_tree(x, bp, *args)
        
        
        
     
    def backtrack_tree(self, sentence, bp, i, j, X):
       #Recurse to get the parse tree
        if i == j:
            return [X, sentence[i]]
        else:
            Y, Z, s = bp[i, j, X]
            return [X, self.backtrack_tree(sentence, bp, i, s, Y), 
                       self.backtrack_tree(sentence, bp, s+1, j, Z)]
    

In [ ]:
parser = PCFGParser(pcfg)

In [ ]:
parser.parse('the dog saw the man with the telescope')

['S', ['NP', ['DT', 'the'], ['NN', 'dog']], ['VP', ['Vt', 'saw'], ['NP', ['NP', ['DT', 'the'], ['NN', 'man']], ['PP', ['P', 'with'], ['NP', ['DT', 'the'], ['NN', 'telescope']]]]]]
S
├── NP
│   ├── DT
│   │   └── the
│   └── NN
│       └── dog
└── VP
    ├── NP
    │   ├── NP
    │   │   ├── DT
    │   │   │   └── the
    │   │   └── NN
    │   │       └── man
    │   └── PP
    │       ├── NP
    │       │   ├── DT
    │       │   │   └── the
    │       │   └── NN
    │       │       └── telescope
    │       └── P
    │           └── with
    └── Vt
        └── saw

